## Imports

In [1]:
%load_ext autoreload
%autoreload 2

# Standard imports
import glob

# 3rd party imports
import cv2
import matplotlib.pyplot as plt
import numpy as np 
from pprint import pprint
import SimpleITK as sitk
sitk.ProcessObject_SetGlobalWarningDisplay(False)
from scipy import ndimage

## Functions

In [2]:
### Functions for IO operations
def load_channels(filepath: str, idx: int) -> np.ndarray:
    """
    Load the image channels from the given filepath.
    
    Args:
        filepath: str, path to the image
        idx: int, index of the image to load
        
    Returns:
        channels: np.ndarray, image channels
    """
    filepaths = sorted(glob.glob(filepath))
    print(f"Found {int(len(filepaths)/2)} slices")
    pprint(filepaths)
    
    # Load the image
    file_idx = idx * 2  # Multiply by 2 because we have 2 channels and they're stored in pairs
    curr_img = (read_tif(filepaths[file_idx]), read_tif(filepaths[file_idx + 1]))
    print("\nCh1:", filepaths[file_idx])
    print("Ch2:", filepaths[file_idx + 1])
    curr_ch1 = curr_img[0]
    curr_ch2 = curr_img[1]

    # Check image stats
    print(f"\nChannel shape: {curr_ch1.shape}")
    print(f"Channel dtype: {curr_ch1.dtype}")
    print(f"Channel 1 min: {curr_ch1.min()}")
    print(f"Channel 1 max: {curr_ch1.max()}")
    print(f"Channel 1 mean: {curr_ch1.mean()}")
    
    return curr_ch1, curr_ch2


def read_tif(filepath):
    """
    Read tiff files using SimpleITK
    
    Args:
        filepath: str, path to tiff file
        
    Returns:
        image: np.ndarray, tiff image
    """
    image = sitk.ReadImage(filepath)
    image = sitk.GetArrayFromImage(image)
    return image


def auto_contrast(data: np.ndarray, alpha: float = None, beta: float = None) -> np.ndarray:
    """
    Preprocess tiff files to automatically adjust brightness and contrast.
    https://stackoverflow.com/questions/56905592/automatic-contrast-and-brightness-adjustment-of-a-color-photo-of-a-sheet-of-pape
    """
    if not alpha:
        alpha = np.iinfo(data.dtype).max / (np.max(data) - np.min(data))
    if not beta:
        beta = -np.min(data) * alpha
    img = cv2.convertScaleAbs(data.copy(), alpha=alpha, beta=beta)
    return img


def gamma_correction(image: np.ndarray, gamma: float=2.0, min_value=None, max_value=None) -> np.ndarray:
    """
    Apply gamma correction to the image.
    
    Args:
        image: np.ndarray, input image
        gamma: float, gamma value
        
    Returns:
        image_enhanced: np.ndarray, gamma corrected image
    """
    if min_value is not None:
        image = image.copy()
        image[image < min_value] = 0
    if max_value is None:
        max_value = image.max()
    else:
        image = image.copy()
        image[image > max_value] = max_value
    # Normalize the image to the range [0, 1]
    image_normalized = image / max_value
    # Apply the exponential transformation
    image_enhanced = np.power(image_normalized, gamma)
    # Rescale the image back to the original intensity range
    image_enhanced = image_enhanced * max_value
    return image_enhanced


def save_figure(image, filename, contours=None):
    """
    Save figure to disk.
    
    Args:
        image: np.ndarray, input image
        filename: str, path to save the image
        contours: np.ndarray, contours to overlay on the image
    """
    plt.figure(figsize=(20, 20))
    plt.imshow(image, cmap='gray')
    if contours is not None:
        plt.contour(contours, colors='red', linewidths=0.15, alpha=0.35)
    plt.axis('off')
    plt.savefig(filename, dpi=600, bbox_inches='tight')
    print(f"Saved figure to {filename}")
    
    
def show(image: np.ndarray, contour: np.ndarray = None,
         image2: np.ndarray = None, contour2: np.ndarray = None, contour_alpha: float = 0.75,
         title: str = "", title2: str = "", 
         xlim: tuple[int, int] = None, ylim: tuple[int, int] = None,
         xlim2: tuple[int, int] = None, ylim2: tuple[int, int] = None,
         axis: bool = True,
         figsize: tuple[int, int] = (10, 10)):
    """
    Display the image.
    
    Args:
        image: np.ndarray, input image
        title: str, title of the image
    """
    f = plt.figure(figsize=figsize)
    # If there are two images, display them side by side
    if image2 is not None:
        plt.subplot(1, 2, 1)
        plt.imshow(image, cmap='gray')
        plt.title(title)
        if contour is not None:
            plt.contour(contour, colors='red', linewidths=0.5, alpha=contour_alpha)
        if xlim is not None:
            plt.xlim(xlim)
        if ylim is not None:
            plt.ylim(ylim)
        plt.axis(axis)
        plt.subplot(1, 2, 2)
        plt.imshow(image2, cmap='gray')
        plt.title(title2)
        if contour2 is not None:
            plt.contour(contour2, colors='red', linewidths=0.5, alpha=contour_alpha)
        if xlim2 is not None:
            plt.xlim(xlim2)
        if ylim2 is not None:
            plt.ylim(ylim2)
        plt.axis(axis)
    # If there is only one image, display it
    else:
        plt.imshow(image, cmap='gray')
        plt.title(title)
        if contour is not None:
            plt.contour(contour, colors='red', linewidths=0.5, alpha=contour_alpha)
        if xlim is not None:
            plt.xlim(xlim)
        if ylim is not None:
            plt.ylim(ylim)
        plt.axis(axis)
    plt.show()
    f.clear()
    plt.close(f)

In [3]:
### Functions for vessel detection
import itk
import numpy as np
from skimage.morphology import remove_small_objects, binary_closing, disk, remove_small_holes

# Parameters for vessel detection
ALPHA = 0.5  # Default 0.5
BETA = 0.5  # Default 1
GAMMA = 5.0  # Default 5

def detect_vessels(input_image: np.ndarray, min_sigma: float=1.0, max_sigma: float=10.0, num_steps: int=10):
    """
    Use the Hessian-based vesselness filter to detect vessels in the image.
    
    Args:
        input_image: np.ndarray, input image
        min_sigma: float, minimum sigma value
        max_sigma: float, maximum sigma value
        num_steps: int, number of steps
        
    Returns:
        segmented_vessels_array: np.ndarray, segmented vessels
    """
    # Run ITK
    input_image = itk.image_from_array(input_image)
    #input_image = itk.imread(input_image, itk.F)

    ImageType = type(input_image)
    Dimension = input_image.GetImageDimension()
    HessianPixelType = itk.SymmetricSecondRankTensor[itk.D, Dimension]
    HessianImageType = itk.Image[HessianPixelType, Dimension]

    objectness_filter = itk.HessianToObjectnessMeasureImageFilter[
        HessianImageType, ImageType
    ].New()
    objectness_filter.SetBrightObject(False)  # Set to True if the structures are bright on a dark background
    objectness_filter.SetScaleObjectnessMeasure(False)  # Set to True to scale the objectness measure by the scale
    objectness_filter.SetAlpha(ALPHA)  # Sensitivity to blob-like structures
                                     # Set/Get Alpha, the weight corresponding to R_A 
                                     # (the ratio of the smallest eigenvalue that has to be large to the larger ones). 
                                     # Smaller values lead to increased sensitivity to the object dimensionality.
    objectness_filter.SetBeta(BETA)   # Sensitivity to plate-like structures - 1.0 default
                                     # Set/Get Beta, the weight corresponding to R_B 
                                     # (the ratio of the largest eigenvalue that has to be small to the larger ones). 
                                     # Smaller values lead to increased sensitivity to the object dimensionality.
    objectness_filter.SetGamma(GAMMA)  # Sensitivity to noise - 5.0 default
                                     # Set/Get Gamma, the weight corresponding to S 
                                     # (the Frobenius norm of the Hessian matrix, or second-order structureness)

    multi_scale_filter = itk.MultiScaleHessianBasedMeasureImageFilter[
        ImageType, HessianImageType, ImageType
    ].New()
    multi_scale_filter.SetInput(input_image)
    multi_scale_filter.SetHessianToMeasureFilter(objectness_filter)
    multi_scale_filter.SetSigmaStepMethodToLogarithmic()
    multi_scale_filter.SetSigmaMinimum(min_sigma)
    multi_scale_filter.SetSigmaMaximum(max_sigma)
    multi_scale_filter.SetNumberOfSigmaSteps(num_steps)

    OutputPixelType = itk.UC
    OutputImageType = itk.Image[OutputPixelType, Dimension]

    rescale_filter = itk.RescaleIntensityImageFilter[ImageType, OutputImageType].New()
    rescale_filter.SetInput(multi_scale_filter)
    rescale_filter.Update()

    # Get numpy array
    segmented_vessels = rescale_filter.GetOutput()
    segmented_vessels_array = itk.array_view_from_image(segmented_vessels)
    segmented_vessels_array = np.asarray(segmented_vessels_array, dtype=np.float32)
    return segmented_vessels_array


def process_vessels(vessel_image: np.ndarray, thresh: int, min_size: int=10, area_threshold: float=2000, smoothing: int=3):
    """
    Process the thresholded vessels.
    
    Args:
        vessel_image: np.ndarray, input image
        thresh: int, threshold value
        min_size: int, minimum size
        area_threshold: float, area threshold
        smoothing: int, smoothing factor
        
    Returns:
        thresholded_vessels: np.ndarray, thresholded vessels
    """
    # Process the thresholded vessels
    thresholded_vessels = vessel_image > thresh
    thresholded_vessels = np.invert(thresholded_vessels)

    # Get rid of small objects
    thresholded_vessels = remove_small_objects(thresholded_vessels, min_size=min_size)
    thresholded_vessels = remove_small_holes(thresholded_vessels, area_threshold=area_threshold)

    # Smoothen edges
    thresholded_vessels = binary_closing(thresholded_vessels, footprint=disk(smoothing))
    
    return thresholded_vessels


def get_brain_mask(brain_image, area_threshold=300000, min_size=10000):
    """
    Get the mask of the brain from the image (run before contrast enhancement).
    
    Args:
        brain_image: np.ndarray, input image
        thresh: int, threshold value
        area_threshold: int, area threshold
        
    Returns:
        mask: np.ndarray, mask of the brain
    """
    _, mask = cv2.threshold(brain_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)
    mask = remove_small_holes(mask.astype(bool), area_threshold=area_threshold)
    mask = remove_small_objects(mask, min_size=min_size)
    return mask


In [4]:
### Functions for evaluation
import csv
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from scipy.spatial.distance import hamming

def dice_coefficient(binary_image1, binary_image2, epsilon=1e-10):
    """
    Compute the Dice coefficient between two binary images.
    
    Parameters:
    - binary_image1: First binary image (numpy array).
    - binary_image2: Second binary image (numpy array).
    
    Returns:
    - dice: Dice coefficient.
    """
    intersection = np.sum(binary_image1 * binary_image2)
    size1 = np.sum(binary_image1)
    size2 = np.sum(binary_image2)
    
    dice = (2. * intersection + epsilon) / (size1 + size2 + epsilon)
    return dice


def iou(binary_image1, binary_image2, epsilon=1e-10):
    """
    Compute the Intersection over Union (IoU) between two binary images.
    
    Parameters:
    - binary_image1: First binary image (numpy array).
    - binary_image2: Second binary image (numpy array).
    
    Returns:
    - iou: IoU.
    """
    intersection = np.sum(binary_image1 * binary_image2)
    union = np.sum(binary_image1 + binary_image2)
    
    if union == 0:
        iou = 1.0
    
    iou = (intersection + epsilon) / (union + epsilon)
    return iou


def precision(binary_image1, binary_image2, epsilon=1e-10):
    """
    Compute the precision between two binary images.
    
    Parameters:
    - binary_image1: First binary image (numpy array).
    - binary_image2: Second binary image (numpy array).
    
    Returns:
    - precision: Precision.
    """
    true_positives = np.sum(binary_image1 * binary_image2)
    false_positives = np.sum(binary_image1 * (1 - binary_image2))
    
    precision = (true_positives) / (true_positives + false_positives + epsilon)
    return precision


def recall(binary_image1, binary_image2, epsilon=1e-10):
    """
    Compute the recall between two binary images.
    
    Parameters:
    - binary_image1: First binary image (numpy array).
    - binary_image2: Second binary image (numpy array).
    
    Returns:
    - recall: Recall.
    """
    true_positives = np.sum(binary_image1 * binary_image2)
    false_negatives = np.sum((1 - binary_image1) * binary_image2)
    
    recall = true_positives / (true_positives + false_negatives + epsilon)
    return recall


def rand_index(binary_image1, binary_image2):
    """
    Compute the Rand index between two binary images.
    
    Parameters:
    - binary_image1: First binary image (numpy array).
    - binary_image2: Second binary image (numpy array).
    
    Returns:
    - rand_index: Rand index.
    """
    true_positives = np.sum(binary_image1 * binary_image2)
    false_positives = np.sum(binary_image1 * (1 - binary_image2))
    false_negatives = np.sum((1 - binary_image1) * binary_image2)
    true_negatives = np.sum((1 - binary_image1) * (1 - binary_image2))
    
    rand_index = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)
    return rand_index

## Load data

In [ ]:
# IO parameters
filepath = "/media/data/u01/Fig7/7b/M106/*/*.tif"
IDX = 3

# Load the image channels
curr_ch1, curr_ch2 = load_channels(filepath, IDX)
curr_ch1 = curr_ch1.astype(np.float32)
curr_ch2 = curr_ch2.astype(np.float32)

Apply processing on the images and retrieve background mask

In [ ]:
# Ch1 settings
gamma_ch1 = 2  # You can adjust this value to control the contrast enhancement
contrast_alpha_ch1 = 0.0225  # Try 0.15 You can adjust this value to control the brightness enhancement 0.5 default

# Ch1 settings
gamma_ch2 = 2  # You can adjust this value to control the contrast enhancement
contrast_alpha_ch2 = 0.125  # Try 0.15 You can adjust this value to control the brightness enhancement 0.5 default


# No change
contrast_ch1 = curr_ch1
contrast_ch2 = curr_ch2

cc_ch1 = auto_contrast(contrast_ch1, alpha=contrast_alpha_ch1)
cc_ch2 = auto_contrast(contrast_ch2, alpha=contrast_alpha_ch2)

# Compute the original image contrast
#contrast_ch0 = gamma_correction(curr_ch0, gamma=gamma_ch0)
#contrast_ch0 = auto_contrast(contrast_ch0, alpha=contrast_alpha_ch0)
#contrast_ch1 = gamma_correction(curr_ch1, gamma=gamma_ch1)
#contrast_ch1 = auto_contrast(contrast_ch1, alpha=contrast_alpha_ch1)
#contrast_ch2 = gamma_correction(curr_ch2, gamma=gamma_ch2)
#contrast_ch2 = auto_contrast(contrast_ch2, alpha=contrast_alpha_ch2)

#bg_mask = gamma_correction(curr_ch0, gamma=gamma_ch0)
if IDX == 99999: 
    bg_alpha = 0.1
else:
    bg_alpha = 0.5  # 7 all but 3
bg_mask = auto_contrast(curr_ch1, alpha=bg_alpha)  # 7
bg_mask = get_brain_mask(bg_mask, area_threshold=25000)  # 255 default ch0, 150 for ch1

show(cc_ch1, bg_mask, title=f"Section {IDX} ch1 contrast", axis=True)
show(cc_ch2, bg_mask, title=f"Section {IDX} ch2 contrast", axis=True)

In [43]:
# Setup FOVs
fovs = [ 
    # xlim,             ylim
    ( (4000, 5000),     (4000, 5000) ),
    ( (2000, 3000),     (2000, 4000) ),
    ( (2000, 4000),     (4000, 6000) ),
    ( (5000, 6000),     (2000, 4000) ),
    ( (5000, 6000),     (1000, 2000) ),
    ( (5000, 6000),     (4000, 5000) ),
]

fov1_xlim, fov1_ylim = fovs[0][0], fovs[0][1][::-1]
fov2_xlim, fov2_ylim = fovs[1][0], fovs[1][1][::-1]
fov3_xlim, fov3_ylim = fovs[2][0], fovs[2][1][::-1]
fov4_xlim, fov4_ylim = fovs[3][0], fovs[3][1][::-1]
fov5_xlim, fov5_ylim = fovs[4][0], fovs[4][1][::-1]
fov6_xlim, fov6_ylim = fovs[5][0], fovs[5][1][::-1]

## Threshold for CH1 and CH2

In [44]:
# Create a threshold mask for the image
curr_ch1_median = ndimage.median_filter(curr_ch1.copy(), size=5)
curr_ch2_median = ndimage.median_filter(curr_ch2.copy(), size=5)  # Repeat for ch2

# Create auto contrast brightened images
auto_ch1 = auto_contrast(curr_ch1, alpha=0.0225)
auto_ch2 = auto_contrast(curr_ch2, alpha=0.125)
auto_ch1_median = ndimage.median_filter(auto_ch1.copy(), size=5) # 5
auto_ch2_median = ndimage.median_filter(auto_ch2.copy(), size=5)

Gamma test

In [ ]:
# Ch1 settings
gamma_ch1 = 2  # You can adjust this value to control the contrast enhancement
contrast_alpha_ch1 = 0.0225  # 2

# Ch2 settings
gamma_ch2 = 2  # You can adjust this value to control the contrast enhancement
contrast_alpha_ch2 = 0.125  # Try 0.15 You can adjust this value to control the brightness enhancement 0.5 default

THRESH = 1750   # Default 1500
THRESH2 = 400  # Default 400
max_value = 2000

if IDX == 0:
     THRESH = 1400  # 1600
     max_value = 4000  # 4000
elif IDX == 1:
     THRESH = 1400  # 1400
     max_value = 4000  # 4000
elif IDX == 2:
     THRESH = 1000  # 1400
     max_value = 4000  # 4000
elif IDX == 3:
     THRESH = 800  # 800
     max_value = 4000

# Create contrast enhanced images
cc_ch1 = gamma_correction(curr_ch1, gamma=gamma_ch1)
cc_ch1 = auto_contrast(cc_ch1, alpha=contrast_alpha_ch1)
cc_ch1_alt = gamma_correction(curr_ch1_median, gamma=gamma_ch1, max_value=max_value)
print("Max value:", cc_ch1_alt.max())
print("Median value:", np.median(cc_ch1_alt))
#cc_ch1_alt = auto_contrast(cc_ch1_alt, alpha=contrast_alpha_ch1)
cc_ch2 = gamma_correction(curr_ch2, gamma=gamma_ch2)
cc_ch2 = auto_contrast(cc_ch2, alpha=contrast_alpha_ch2)



print(f"Threshold for ch1: {THRESH}")
print(f"Threshold for ch2: {THRESH2}")
curr_ch1_thresh = cc_ch1_alt.copy() > THRESH
curr_ch1_thresh[curr_ch1_thresh != 0] = 1
curr_ch1_thresh = curr_ch1_thresh.astype(bool)

curr_ch2_thresh = curr_ch2_median.copy() > THRESH2
curr_ch2_thresh[curr_ch2_thresh != 0] = 1
curr_ch2_thresh = curr_ch2_thresh.astype(bool)

##########################################
# FOV 1
##########################################

"""
show(image=cc_ch1, title=f"Section {IDX} ch1 contrast FOV1", 
     xlim=fov1_xlim, ylim=fov1_ylim,
     image2=cc_ch2, title2=f"Section {IDX} ch2 contrast FOV1", 
     xlim2=fov1_xlim, ylim2=fov1_ylim)
"""

show(image=cc_ch1_alt, title=f"Section {IDX} ch1 contrast alt FOV1", 
     xlim=fov1_xlim, ylim=fov1_ylim,
     contour=curr_ch1_thresh, 
     image2=auto_ch2, title2=f"Section {IDX} ch2 contrast FOV1", 
     contour2=curr_ch2_thresh, 
     xlim2=fov1_xlim, ylim2=fov1_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV1", 
     xlim=fov1_xlim, ylim=fov1_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV1", 
     xlim2=fov1_xlim, ylim2=fov1_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV1", 
     contour=curr_ch1_thresh, 
     xlim=fov1_xlim, ylim=fov1_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV1", 
     contour2=curr_ch2_thresh, 
     xlim2=fov1_xlim, ylim2=fov1_ylim)

print("##########################################")

##########################################
# FOV 2
##########################################

"""
show(image=cc_ch1, title=f"Section {IDX} ch1 contrast FOV2", 
     contour=curr_ch1_thresh, 
     xlim=fov2_xlim, ylim=fov2_ylim,
     image2=cc_ch2, title2=f"Section {IDX} ch2 contrast FOV2", 
     contour2=curr_ch2_thresh, 
     xlim2=fov2_xlim, ylim2=fov2_ylim)
"""

show(image=cc_ch1_alt, title=f"Section {IDX} ch1 contrast alt FOV2", 
     contour=curr_ch1_thresh, 
     xlim=fov2_xlim, ylim=fov2_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 contrast FOV2", 
     contour2=curr_ch2_thresh, 
     xlim2=fov2_xlim, ylim2=fov2_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV2", 
     xlim=fov2_xlim, ylim=fov2_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV2", 
     xlim2=fov2_xlim, ylim2=fov2_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV2", 
     contour=curr_ch1_thresh, 
     xlim=fov2_xlim, ylim=fov2_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV2", 
     contour2=curr_ch2_thresh, 
     xlim2=fov2_xlim, ylim2=fov2_ylim)

print("##########################################")

##########################################
# FOV 3
##########################################

show(image=cc_ch1_alt, title=f"Section {IDX} ch1 contrast alt FOV3", 
     contour=curr_ch1_thresh, 
     xlim=fov3_xlim, ylim=fov3_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 contrast FOV3", 
     contour2=curr_ch2_thresh, 
     xlim2=fov3_xlim, ylim2=fov3_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV3",
     xlim=fov3_xlim, ylim=fov3_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV3", 
     xlim2=fov3_xlim, ylim2=fov3_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV3", 
     contour=curr_ch1_thresh,
     xlim=fov3_xlim, ylim=fov3_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV3", 
     contour2=curr_ch2_thresh, 
     xlim2=fov3_xlim, ylim2=fov3_ylim)

print("##########################################")

##########################################
# FOV 4
##########################################

show(image=cc_ch1_alt, title=f"Section {IDX} ch1 contrast alt FOV4", 
     contour=curr_ch1_thresh, 
     xlim=fov4_xlim, ylim=fov4_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 contrast FOV4", 
     contour2=curr_ch2_thresh, 
     xlim2=fov4_xlim, ylim2=fov4_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV4",
     xlim=fov4_xlim, ylim=fov4_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV4", 
     xlim2=fov4_xlim, ylim2=fov4_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV4", 
     contour=curr_ch1_thresh,
     xlim=fov4_xlim, ylim=fov4_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV4", 
     contour2=curr_ch2_thresh, 
     xlim2=fov4_xlim, ylim2=fov4_ylim)

print("##########################################")

##########################################
# FOV 5
##########################################

show(image=cc_ch1_alt, title=f"Section {IDX} ch1 contrast alt FOV5", 
     contour=curr_ch1_thresh, 
     xlim=fov5_xlim, ylim=fov5_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV5", 
     contour2=curr_ch2_thresh, 
     xlim2=fov5_xlim, ylim2=fov5_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV5",
     xlim=fov5_xlim, ylim=fov5_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV5", 
     xlim2=fov5_xlim, ylim2=fov5_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV5", 
     contour=curr_ch1_thresh,
     xlim=fov5_xlim, ylim=fov5_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV5", 
     contour2=curr_ch2_thresh, 
     xlim2=fov5_xlim, ylim2=fov5_ylim)

print("##########################################")

##########################################
# FOV 6
##########################################

show(image=cc_ch1_alt, title=f"Section {IDX} ch1 contrast alt FOV6", 
     contour=curr_ch1_thresh, 
     xlim=fov6_xlim, ylim=fov6_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 contrast FOV6", 
     contour2=curr_ch2_thresh, 
     xlim2=fov6_xlim, ylim2=fov6_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV6",
     xlim=fov6_xlim, ylim=fov6_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV6", 
     xlim2=fov6_xlim, ylim2=fov6_ylim)

show(image=auto_ch1, title=f"Section {IDX} ch1 brightened FOV6", 
     contour=curr_ch1_thresh,
     xlim=fov6_xlim, ylim=fov6_ylim,
     image2=auto_ch2, title2=f"Section {IDX} ch2 brightened FOV6", 
     contour2=curr_ch2_thresh, 
     xlim2=fov6_xlim, ylim2=fov6_ylim)

print("##########################################")

##########################################################
# FULL SECTION
##########################################################

show(image=cc_ch1, title=f"Full section {IDX} ch1 contrast", 
     image2=cc_ch2, title2=f"Full section {IDX} ch2 contrast",
     figsize=(20, 10), axis=True)

show(image=auto_ch1, title=f"Full section {IDX} ch1 brightened", 
     image2=auto_ch2, title2=f"Full section {IDX} ch2 brightened",
     figsize=(20, 10), axis=True)

show(image=curr_ch1_thresh, title=f"Full section {IDX} ch1 thresholded", 
     image2=curr_ch2_thresh, title2=f"Full section {IDX} ch2 thresholded",
     figsize=(20, 10), axis=True)

#thresholded_vessels_ch1 = curr_ch1_thresh
#thresholded_vessels_ch2 = curr_ch2_thresh

## Hessian Filter
https://examples.itk.org/src/nonunit/review/segmentbloodvesselswithmultiscalehessianbasedmeasure/documentation

# CHANNEL 1

In [ ]:
# Parameters for vessel detection
sigma_minimum = 1.0  # Range of scales in which MultiScaleHessianBasedMeasureImageFilter will search for vessels
sigma_maximum = 10.0  # 10
number_of_sigma_steps = 10  # 10 Number of scales to search for vessels

# Parameters for post-processing
thresh = 230  # Threshold for binarization, 230 and 25 and 15
min_size = 100  # Minimum size of objects to keep 100
area_threshold = 2000 # Minimum area of holes to keep
smoothing = 1  # Smoothing factor for closing, 3

#############################################################

# Alternative: load image in memory
input_image = auto_ch1_median * bg_mask
#input_image = cc_ch1_alt * bg_mask
input_image = input_image.astype(np.float32)
input_image *= 255.0

show(input_image, title="CH1: Input image", axis=False)

# Print statistics
print("Input image type:", input_image.dtype)
print("Input image min:", input_image.min())
print("Input image max:", input_image.max())

# Run the vessel detection
segmented_vessels_array = detect_vessels(input_image, sigma_minimum, sigma_maximum, number_of_sigma_steps)

# Process the thresholded vessels
thresholded_vessels_ch1 = process_vessels(segmented_vessels_array, thresh=thresh, min_size=min_size, area_threshold=area_threshold, smoothing=smoothing)
thresholded_vessels_ch1 = thresholded_vessels_ch1 * bg_mask * curr_ch1_thresh

# Print statistics
print("Vesselness image statistics:")
print("Shape:", segmented_vessels_array.shape)
print("Min:", segmented_vessels_array.min())
print("Max:", segmented_vessels_array.max())
print("Mean:", segmented_vessels_array.mean())
print("Median:", np.median(segmented_vessels_array))
#print("Std:", segmented_vessels_array.std())

print("CHANNEL 1")

# Plot the raw vesselness image
show(image=segmented_vessels_array, title=f"CH1: Vesselness image",
     image2=thresholded_vessels_ch1, title2=f"CH1: Vessel mask",
     axis=False)

# Show the results
show(image=input_image, title="CH1: Input image",
     image2=input_image, title2="Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     axis=False)

# Plot the raw vesselness image FOV1
show(image=segmented_vessels_array, title=f"CH1 FOV1: Vesselness image",
     image2=thresholded_vessels_ch1, title2=f"CH1 FOV1: Vessel mask",
     xlim=fov1_xlim, ylim=fov1_ylim, xlim2=fov1_xlim, ylim2=fov1_ylim,
     axis=False)


# FOV1
show(image=input_image, title="CH1 FOV1: Input image",
     image2=input_image, title2="CH1 FOV1: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     xlim=fov1_xlim, ylim=fov1_ylim, xlim2=fov1_xlim, ylim2=fov1_ylim,
     axis=False)

# FOV2
show(image=input_image, title="CH1 FOV2: Input image",
     image2=input_image, title2="CH1 FOV2: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     xlim=fov2_xlim, ylim=fov2_ylim, xlim2=fov2_xlim, ylim2=fov2_ylim,
     axis=False)

# FOV3
show(image=input_image, title="CH1 FOV3: Input image",
     image2=input_image, title2="CH1 FOV3: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     xlim=fov3_xlim, ylim=fov3_ylim, xlim2=fov3_xlim, ylim2=fov3_ylim,
     axis=False)

# FOV4
show(image=input_image, title="CH1 FOV4: Input image",
     image2=input_image, title2="CH1 FOV4: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     xlim=fov4_xlim, ylim=fov4_ylim, xlim2=fov4_xlim, ylim2=fov4_ylim,
     axis=False)

# FOV5
show(image=input_image, title="CH1 FOV5: Input image",
     image2=input_image, title2="CH1 FOV5: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     xlim=fov5_xlim, ylim=fov5_ylim, xlim2=fov5_xlim, ylim2=fov5_ylim,
     axis=False)

# FOV6
show(image=input_image, title="CH1 FOV6: Input image",
     image2=input_image, title2="CH1 FOV6: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch1, contour_alpha=0.45,
     xlim=fov6_xlim, ylim=fov6_ylim, xlim2=fov6_xlim, ylim2=fov6_ylim,
     axis=False)

# Full section
show(image=thresholded_vessels_ch1, title="CH1: Vessel segmentation",
     image2=curr_ch2_thresh, title2="CH2: Thresholded vessels",
     figsize=(20, 10), axis=False)

# CHANNEL 2

In [ ]:
# Parameters for vessel detection
sigma_minimum = 1.0  # Range of scales in which MultiScaleHessianBasedMeasureImageFilter will search for vessels
sigma_maximum = 10.0  # 10
number_of_sigma_steps = 10  # 10 Number of scales to search for vessels

# Parameters for post-processing
thresh = 230  # Threshold for binarization, 230
min_size = 100  # Minimum size of objects to keep
area_threshold = 2000 # Minimum area of holes to keep
smoothing = 1  # Smoothing factor for closing, 3

#############################################################

# Alternative: load image in memory
input_image = auto_ch2_median * bg_mask
input_image = input_image.astype(np.float32)
input_image *= 255.0

show(input_image, title="CH2: Input image", axis=False)

# Print statistics
print("Input image type:", input_image.dtype)
print("Input image min:", input_image.min())
print("Input image max:", input_image.max())

# Run the vessel detection
segmented_vessels_array = detect_vessels(input_image, sigma_minimum, sigma_maximum, number_of_sigma_steps)

# Process the thresholded vessels
thresholded_vessels_ch2 = process_vessels(segmented_vessels_array, thresh=thresh, min_size=min_size, area_threshold=area_threshold, smoothing=smoothing)
thresholded_vessels_ch2 = thresholded_vessels_ch2 * bg_mask * curr_ch2_thresh

# Print statistics
print("Vesselness image statistics:")
print("Shape:", segmented_vessels_array.shape)
print("Min:", segmented_vessels_array.min())
print("Max:", segmented_vessels_array.max())
print("Mean:", segmented_vessels_array.mean())
print("Median:", np.median(segmented_vessels_array))
#print("Std:", segmented_vessels_array.std())

print("CHANNEL 2")

# Plot the raw vesselness image
show(image=segmented_vessels_array, title=f"CH2: Vesselness image",
     image2=thresholded_vessels_ch2, title2=f"CH2: Vessel mask",
     axis=False)

# Show the results
show(image=input_image, title="CH2: Input image",
     image2=input_image, title2="Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     axis=False)

# Plot the raw vesselness image FOV1
show(image=segmented_vessels_array, title=f"CH2 FOV1: Vesselness image",
     image2=thresholded_vessels_ch2, title2=f"CH2 FOV1: Vessel mask",
     xlim=fov1_xlim, ylim=fov1_ylim, xlim2=fov1_xlim, ylim2=fov1_ylim,
     axis=False)


# FOV1
show(image=input_image, title="CH2 FOV1: Input image",
     image2=input_image, title2="CH2 FOV1: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     xlim=fov1_xlim, ylim=fov1_ylim, xlim2=fov1_xlim, ylim2=fov1_ylim,
     axis=False)

# FOV2
show(image=input_image, title="CH2 FOV2: Input image",
     image2=input_image, title2="CH2 FOV2: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     xlim=fov2_xlim, ylim=fov2_ylim, xlim2=fov2_xlim, ylim2=fov2_ylim,
     axis=False)

# FOV3
show(image=input_image, title="CH2 FOV3: Input image",
     image2=input_image, title2="CH2 FOV3: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     xlim=fov3_xlim, ylim=fov3_ylim, xlim2=fov3_xlim, ylim2=fov3_ylim,
     axis=False)

# FOV4
show(image=input_image, title="CH2 FOV4: Input image",
     image2=input_image, title2="CH2 FOV4: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     xlim=fov4_xlim, ylim=fov4_ylim, xlim2=fov4_xlim, ylim2=fov4_ylim,
     axis=False)

# FOV5
show(image=input_image, title="CH2 FOV5: Input image",
     image2=input_image, title2="CH2 FOV5: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     xlim=fov5_xlim, ylim=fov5_ylim, xlim2=fov5_xlim, ylim2=fov5_ylim,
     axis=False)

# FOV6
show(image=input_image, title="CH2 FOV6: Input image",
     image2=input_image, title2="CH2 FOV6: Vessel mask contours over input image",
     contour2=thresholded_vessels_ch2, contour_alpha=0.45,
     xlim=fov6_xlim, ylim=fov6_ylim, xlim2=fov6_xlim, ylim2=fov6_ylim,
     axis=False)

# Full section
show(image=thresholded_vessels_ch1, title="CH1: Vessel segmentation",
     image2=thresholded_vessels_ch2, title2="CH2: Vessel segmentation",
     figsize=(10, 10), axis=False)

## Compute statistics 

i = 0:


Dice coefficient: 0.42411068987180006
IoU score: 0.2691246695729526
Precision score: 0.6294159745347224
Recall score: 0.31979782500429416
SSIM score: 0.7485813152339447
MSE score: 0.11598956526140992
Hamming distance: 0.11598956526140992
Rand index: 0.8840104347385901

i = 1:

Dice coefficient: 0.42784078682519283
IoU score: 0.27213578830938767
Precision score: 0.6226656435973017
Recall score: 0.3258774979835265
SSIM score: 0.7482098056616051
MSE score: 0.11578135242146209
Hamming distance: 0.11578135242146209
Rand index: 0.884218647578538


i = 2:

Dice coefficient: 0.35917833330468135
IoU score: 0.2189015056268004
Precision score: 0.5407998576777105
Recall score: 0.26887851932163065
SSIM score: 0.8319550597966561
MSE score: 0.07499218512130308
Hamming distance: 0.07499218512130308
Rand index: 0.9250078148786969

i = 3:

Dice coefficient: 0.3879609013801778
IoU score: 0.24066469709843755
Precision score: 0.612943451204528
Recall score: 0.2837936685726384
SSIM score: 0.8111572870712608
MSE score: 0.08667642593981273
Hamming distance: 0.08667642593981273
Rand index: 0.9133235740601873

In [ ]:
thresholded_ch0 = thresholded_vessels_ch1
thresholded_ch1 = thresholded_vessels_ch2

# Compute the metrics
dice_score = dice_coefficient(thresholded_ch0, thresholded_ch1)  
iou_score = iou(thresholded_ch0, thresholded_ch1)  # Strongly penalizes over-segmentation and under-segmentation
precision_score = precision(thresholded_ch0, thresholded_ch1) 
recall_score = recall(thresholded_ch0, thresholded_ch1)
ssim_score = ssim(thresholded_ch0, thresholded_ch1)
mse_score = mean_squared_error(thresholded_ch0, thresholded_ch1)
thresh_ch0_flat = thresholded_ch0.flatten()
thresh_ch1_flat = thresholded_ch1.flatten()
hamming_distance = hamming(thresh_ch0_flat, thresh_ch1_flat)
rand_score = rand_index(thresholded_ch0, thresholded_ch1)  # Measures how close points are clustered together

print("idx =", IDX)
print("thresh =", THRESH)
print("Dice coefficient:", dice_score)
print("IoU score:", iou_score)
print("Precision score:", precision_score)
print("Recall score:", recall_score)
print("SSIM score:", ssim_score)
print("MSE score:", mse_score)
print("Hamming distance:", hamming_distance)
print("Rand index:", rand_score)

# Write the solutions to a CSV file
csv_filename = 'stats_enhanced_M106.csv'

# Write to rows
rows = [["Index", "Dice coefficient", "IoU score", "Precision", "Recall", "SSIM", "MSE", "Hamming distance", "Rand index"]]
rows.append([IDX, dice_score, iou_score, precision_score, recall_score, ssim_score, mse_score, hamming_distance, rand_score])

#with open(csv_filename, mode='w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerows(rows)

M106

i = 0 (original 2200)

no mask
Dice coefficient: 0.47672714941402217
IoU score: 0.31296241460000623
Precision score: 0.37038620546108075
Recall score: 0.6687230776390751
SSIM score: 0.6041838404919053
MSE score: 0.19605965989461135
Hamming distance: 0.19605965989461135
Rand index: 0.8039403401053886

thresh = 1000
Dice coefficient: 0.5025460148664612
IoU score: 0.33560030548895003
Precision score: 0.4298795843696149
Recall score: 0.6047769228407175
SSIM score: 0.6562935647078479
MSE score: 0.15990274121342082
Hamming distance: 0.15990274121342082
Rand index: 0.8400972587865791

thresh = 1200
Dice coefficient: 0.5075368376586201
IoU score: 0.3400665761574946
Precision score: 0.472912310169995
Recall score: 0.547632007730418
SSIM score: 0.6872432469474905
MSE score: 0.14193144387319145
Hamming distance: 0.14193144387319145
Rand index: 0.8580685561268085

**thresh = 1400**
Dice coefficient: 0.5011254601422405
IoU score: 0.33433449352592
Precision score: 0.5191762362304143
Recall score: 0.48428769274467265
SSIM score: 0.7120537016214114
MSE score: 0.1287751076586028
Hamming distance: 0.1287751076586028
Rand index: 0.8712248923413972

thresh = 1600
Dice coefficient: 0.4814350779093821
IoU score: 0.31703292424704993
Precision score: 0.5675273750605996
Recall score: 0.4180222806076477
SSIM score: 0.7317205760996874
MSE score: 0.12026754696657609
Hamming distance: 0.12026754696657609
Rand index: 0.879732453033424

thresh = 1800
Dice coefficient: 0.4583110526190515
IoU score: 0.2972785485669073
Precision score: 0.5971730846358245
Recall score: 0.3718450639419566
SSIM score: 0.741116494168267
MSE score: 0.11739112952705186
Hamming distance: 0.11739112952705186
Rand index: 0.8826088704729481

i = 1 (original 2000)

no mask
Dice coefficient: 0.46782207723345515
IoU score: 0.30533143069229324
Precision score: 0.3616159525834917
Recall score: 0.6623548023468838
SSIM score: 0.5944701695120423
MSE score: 0.200178008456473
Hamming distance: 0.200178008456473
Rand index: 0.799821991543527

thresh = 1000
Dice coefficient: 0.5042112707056357
IoU score: 0.3370872241720236
Precision score: 0.4683638278556711
Recall score: 0.5460008660540057
SSIM score: 0.6840184170844298
MSE score: 0.14263528221310992
Hamming distance: 0.14263528221310992
Rand index: 0.85736471778689

thresh = 1200
Dice coefficient: 0.49797427359951096
IoU score: 0.33153511610808106
Precision score: 0.5195630878441235
Recall score: 0.47810799566564377
SSIM score: 0.7128161258855013
MSE score: 0.12805442984942703
Hamming distance: 0.12805442984942703
Rand index: 0.871945570150573

**thresh = 1400**
Dice coefficient: 0.48162972627177847
IoU score: 0.31720176205055706
Precision score: 0.5629169305311986
Recall score: 0.42085654612199963
SSIM score: 0.7306963022624658
MSE score: 0.12034011585262294
Hamming distance: 0.12034011585262294
Rand index: 0.8796598841473771

thresh = 1600
Dice coefficient: 0.4595908083781006
IoU score: 0.2983563139442168
Precision score: 0.5960952101069142
Recall score: 0.37395580758890845
SSIM score: 0.7412197757823129
MSE score: 0.11682107778963834
Hamming distance: 0.11682107778963834
Rand index: 0.8831789222103616

thresh = 1800
Dice coefficient: 0.44163626507058085
IoU score: 0.2833974220341975
Precision score: 0.6134206037515663
Recall score: 0.34501665707670104
SSIM score: 0.745937567421992
MSE score: 0.11588896169018616
Hamming distance: 0.11588896169018616
Rand index: 0.8841110383098139

i = 2 (original 2000)

no mask
Dice coefficient: 0.36291282283738036
IoU score: 0.22168203862324337
Precision score: 0.24685436458457544
Recall score: 0.6849343897191634
SSIM score: 0.6460981096769284
MSE score: 0.18796559629673285
Hamming distance: 0.18796559629673285
Rand index: 0.8120344037032672


**thresh = 1000**
Dice coefficient: 0.4165464264858243
IoU score: 0.26306197633655803
Precision score: 0.5039227428138098
Recall score: 0.354993322750563
SSIM score: 0.8215916080427953
MSE score: 0.07773129792460191
Hamming distance: 0.07773129792460191
Rand index: 0.9222687020753981


thresh = 1200
Dice coefficient: 0.3987961797614054
IoU score: 0.24906022251557017
Precision score: 0.5192037755366908
Recall score: 0.3237223281021639
SSIM score: 0.8263121923587782
MSE score: 0.07629151536745102
Hamming distance: 0.07629151536745102
Rand index: 0.923708484632549

thresh = 1400
Dice coefficient: 0.38551205929679194
IoU score: 0.23878286704877952
Precision score: 0.5272931777739085
Recall score: 0.30381959067293557
SSIM score: 0.828550361476516
MSE score: 0.07570490430071579
Hamming distance: 0.07570490430071579
Rand index: 0.9242950956992843

thresh = 1600
Dice coefficient: 0.37332748875599187
IoU score: 0.22950377914143721
Precision score: 0.533435625279878
Recall score: 0.2871429221434867
SSIM score: 0.8301766883931667
MSE score: 0.0753497187802167
Hamming distance: 0.0753497187802167
Rand index: 0.9246502812197833

thresh = 1800
Dice coefficient: 0.3652086988775218
IoU score: 0.22339775029801215
Precision score: 0.5374605065780946
Recall score: 0.2765701656239947
SSIM score: 0.8312124879835611
MSE score: 0.07514983427623859
Hamming distance: 0.07514983427623859
Rand index: 0.9248501657237614

thresh = 2000
Dice coefficient: 0.35917833330468135
IoU score: 0.2189015056268004
Precision score: 0.5407998576777105
Recall score: 0.26887851932163065
SSIM score: 0.8319550597966561
MSE score: 0.07499218512130308
Hamming distance: 0.07499218512130308
Rand index: 0.9250078148786969



i = 3 (original 1000)

no mask
Dice coefficient: 0.4322241418857131
IoU score: 0.2756925613114047
Precision score: 0.3214284478114834
Recall score: 0.6595799208240772
SSIM score: 0.6697726538931316
MSE score: 0.16774223049025985
Hamming distance: 0.16774223049025985
Rand index: 0.8322577695097402

thresh = 750
Dice coefficient: 0.4590960968007366
IoU score: 0.29793947295970225
Precision score: 0.49875884849621777
Recall score: 0.4252768471917144
SSIM score: 0.7826364595440669
MSE score: 0.09700527256928453
Hamming distance: 0.09700527256928453
Rand index: 0.9029947274307155

**thresh = 800**
Dice coefficient: 0.45261626874092675
IoU score: 0.2925042183121846
Precision score: 0.5206103101403131
Recall score: 0.4003312164230105
SSIM score: 0.7899104427730833
MSE score: 0.0937320828052005
Hamming distance: 0.0937320828052005
Rand index: 0.9062679171947995

thresh = 900
Dice coefficient: 0.43812354688446187
IoU score: 0.28051101353792685
Precision score: 0.556280270620867
Recall score: 0.3613673141677577
SSIM score: 0.7996764566085222
MSE score: 0.08972225722701395
Hamming distance: 0.08972225722701395
Rand index: 0.910277742772986

thresh = 1000
Dice coefficient: 0.42390931290666267
IoU score: 0.26896251362822654
Precision score: 0.5792684253114408
Recall score: 0.3342609248295756
SSIM score: 0.8050663118161301
MSE score: 0.08794488657223096
Hamming distance: 0.08794488657223096
Rand index: 0.912055113427769

Run the whole thing

## Threshold method (batch)

In [ ]:
from tqdm import tqdm

data_path = "/media/data/u01/Fig7/7b/M106/*/*.tif"
output_ch1_path = "/media/data/u01/lightsheet/quant-fig7/7b-M106 run 2/segmentation/ch1/"
output_ch2_path = "/media/data/u01/lightsheet/quant-fig7/7b-M106 run 2/segmentation/ch2/"
output_csv_ch1_ch2_path = "/media/data/u01/lightsheet/quant-fig7/7b-M106 run 2/stats_enhanced2_ch1_ch2.csv"

################################################################################

# Ch1 settings
gamma_ch1 = 2  # You can adjust this value to control the contrast enhancement
contrast_alpha_ch1 = 0.0225  # 2

# Ch2 settings
gamma_ch2 = 2  # You can adjust this value to control the contrast enhancement
contrast_alpha_ch2 = 0.125  # Try 0.15 You can adjust this value to control the brightness enhancement 0.5 default

# Parameters for vessel detection
sigma_minimum = 1.0  # Range of scales in which MultiScaleHessianBasedMeasureImageFilter will search for vessels
sigma_maximum = 10.0  # 10
number_of_sigma_steps = 10  # 10 Number of scales to search for vessels

# Parameters for post-processing
thresh1 = 230  # Threshold for binarization, 230 (ch1)
thresh2 = 230  # Threshold for binarization, 230 (ch2)
min_size1 = 100  # Minimum size of objects to keep (ch1)
min_size2 = 100  # Minimum size of objects to keep (ch2)
area_threshold = 2000 # Minimum area of holes to keep
smoothing = 1  # Smoothing factor for closing, 3

# Read all tif files in the folder
data_files = sorted(glob.glob(data_path))
num_slices = len(data_files) // 2
rows_ch1_ch2 = [["Index", "Dice coefficient", "IoU score", "Precision", "Recall", "SSIM", "MSE", "Hamming distance", "Rand index"]]

# Load the image channels
for i in tqdm(range(num_slices)):
    curr_ch1, curr_ch2 = load_channels(data_path, i)
    curr_ch1 = curr_ch1.astype(np.float32)
    curr_ch2 = curr_ch2.astype(np.float32)
        
    # Ch1 settings
    gamma_ch1 = 2  # You can adjust this value to control the contrast enhancement
    contrast_alpha_ch1 = 0.0225  # 2

    # Ch2 settings
    gamma_ch2 = 2  # You can adjust this value to control the contrast enhancement
    contrast_alpha_ch2 = 0.125  # Try 0.15 You can adjust this value to control the brightness enhancement 0.5 default

    THRESH = 1750   # Default 1500
    THRESH2 = 400  # Default 400
    max_value = 2000


    if i == 0:
        THRESH = 1400  # 1600
        max_value = 4000  # 4000
    elif i == 1:
        THRESH = 1400  # 1400
        max_value = 4000  # 4000
    elif i == 2:
        THRESH = 1000  # 1400
        max_value = 4000  # 4000
    elif i == 3:
        THRESH = 800  # 800
        max_value = 4000
    
    # Create a threshold mask for the image
    curr_ch1_median = ndimage.median_filter(curr_ch1.copy(), size=5)
    curr_ch2_median = ndimage.median_filter(curr_ch2.copy(), size=5)  # Repeat for ch2
    
    cc_ch1_alt = gamma_correction(curr_ch1_median, gamma=gamma_ch1, max_value=max_value)

    curr_ch1_thresh = cc_ch1_alt.copy() > THRESH
    curr_ch1_thresh[curr_ch1_thresh != 0] = 1
    curr_ch1_thresh = curr_ch1_thresh.astype(bool)
    
    curr_ch2_thresh = curr_ch2_median.copy() > THRESH2
    curr_ch2_thresh[curr_ch2_thresh != 0] = 1
    curr_ch2_thresh = curr_ch2_thresh.astype(bool)

    auto_ch1 = auto_contrast(curr_ch1, alpha=0.0225)
    auto_ch2 = auto_contrast(curr_ch2, alpha=0.125)
    auto_ch1_median = ndimage.median_filter(auto_ch1.copy(), size=5)
    auto_ch2_median = ndimage.median_filter(auto_ch2.copy(), size=5)


    bg_alpha = 0.5  #
    bg_mask = auto_contrast(curr_ch1, alpha=bg_alpha)  # 
    bg_mask = get_brain_mask(bg_mask, area_threshold=25000)  # 255 default ch0, 150 for ch1
    
    input_ch1 = auto_ch1_median * bg_mask
    input_ch1 = input_ch1.astype(np.float32)
    input_ch1 *= 255.0
        
    input_ch2 = auto_ch2_median * bg_mask
    input_ch2 = input_ch2.astype(np.float32)
    input_ch2 *= 255.0


    # Run the vessel detection
    segmented_vessels_ch1 = detect_vessels(input_ch1, sigma_minimum, sigma_maximum, number_of_sigma_steps)
    segmented_vessels_ch2 = detect_vessels(input_ch2, sigma_minimum, sigma_maximum, number_of_sigma_steps)

    # Process the thresholded vessels
    thresholded_vessels_ch1 = process_vessels(segmented_vessels_ch1, thresh=thresh1, min_size=min_size1, area_threshold=area_threshold, smoothing=smoothing)
    thresholded_vessels_ch2 = process_vessels(segmented_vessels_ch2, thresh=thresh2, min_size=min_size2, area_threshold=area_threshold, smoothing=smoothing)

    thresholded_vessels_ch1 = thresholded_vessels_ch1 * bg_mask * curr_ch1_thresh
    thresholded_vessels_ch2 = thresholded_vessels_ch2 * bg_mask * curr_ch2_thresh
    
    # Save to file
    sitk_ch1 = sitk.GetImageFromArray(thresholded_vessels_ch1.astype(np.uint8))  # Ch1
    output_ch1_file = output_ch1_path + f"ch1_seg_{str(i).zfill(4)}.tif"
    sitk.WriteImage(sitk_ch1, output_ch1_file)
    sitk_ch2 = sitk.GetImageFromArray(thresholded_vessels_ch2.astype(np.uint8))  # Ch1
    output_ch2_file = output_ch2_path + f"ch2_seg_{str(i).zfill(4)}.tif"
    sitk.WriteImage(sitk_ch2, output_ch2_file)
    
    # Compute statistics between ch1 and ch2
    thresh_ch1_flat = thresholded_vessels_ch1.flatten()
    thresh_ch2_flat = thresholded_vessels_ch2.flatten()
    dice_score = dice_coefficient(thresholded_vessels_ch1, thresholded_vessels_ch2)  
    iou_score = iou(thresholded_vessels_ch1, thresholded_vessels_ch2)  # Strongly penalizes over-segmentation and under-segmentation
    precision_score = precision(thresholded_vessels_ch1, thresholded_vessels_ch2) 
    recall_score = recall(thresholded_vessels_ch1, thresholded_vessels_ch2)
    ssim_score = ssim(thresholded_vessels_ch1, thresholded_vessels_ch2)
    mse_score = mean_squared_error(thresholded_vessels_ch1, thresholded_vessels_ch2)
    hamming_distance = hamming(thresh_ch1_flat, thresh_ch2_flat)
    rand_score = rand_index(thresholded_vessels_ch1, thresholded_vessels_ch2)  # Measures how close points are clustered together
    rows_ch1_ch2.append([i, dice_score, iou_score, precision_score, recall_score, ssim_score, mse_score, hamming_distance, rand_score])
    print("ch1-ch2:", rows_ch1_ch2[i + 1])

with open(output_csv_ch1_ch2_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows_ch1_ch2)

In [ ]:
"""
i=0 should match:

"""